In [11]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import re

import warnings
warnings.filterwarnings("ignore")

from selenium import webdriver
from bs4 import BeautifulSoup


In [12]:
engine = create_engine('sqlite:///data.db', echo=False)
df = pd.read_sql('data.db', con=engine, index_col='ID')

In [13]:
df.head(3)

,title,category,content
ID,,,
0,피아니스트(로만 폴란스키 영화),2002년 영화홀로코스트 영화실화 바탕 영화폴란드 영화프랑스 영화영국 영화독일 영화...,로만 폴란스키 감독 장편 연출 작품 [ 펼치기 · 접기 ] 피아니스트의 주요 수상...
1,제1차 세계 대전,제1차 세계 대전,제1차 세계 대전 관련 틀 [ 펼치기 · 접기 ]제1차 세계 대전 참전국 [ 펼치기...
2,김영남,최고인민회의 상임위원장북한 외무상조선로동당 국제비서조선로동당 6기 정치국 상무위원조...,은(는) 여기로 연결됩니다. 동명이인에 대한 내용은 김영남(동명이인) 문서를의...


In [14]:
def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    result = hangul.sub('', text)
    return result

In [17]:
print(text_cleaning(df['content'][0][:500]))

로만 폴란스키 감독 장편 연출 작품  펼치기  접기   피아니스트의 주요 수상 이력      펼치기  접기  선정 대 세기 영화   펼치기  접기   년 는 명의 국제 영화 평론가들을 대상으로 사상 최고의 세기 영화 편을 선정했다  위멀홀랜드 드라이브위화양연화위데어 윌 비 블러드위센과 치히로의 행방불명위보이후드위이터널 선샤인위트리 오브 라이프위하나 그리고 둘위씨민과 나데르의 별거위노인을 위한 나라는 없다위인사이드 르윈위조디악위칠드런 오브 맨위액트 오브 킬링위개월 주 그리고 일위홀리 모터스위판의 미로위하얀 리본위매드 맥스 분노의 도로위시네도키 뉴욕위그랜드 부다페스트 호텔위사랑도 통역이 되나요위히든위마스터위메멘토위시위소셜 네트워크위그녀에게위월위올드보이위마가렛위타인의 삶위다크 나이트위사울의 아들위


In [27]:
for column in df.columns:
    column = str(column)
    df[column] = df[column].apply(lambda x : text_cleaning(x))

In [28]:
df.head(3)

,title,category,content
ID,,,
0,피아니스트로만 폴란스키 영화,년 영화홀로코스트 영화실화 바탕 영화폴란드 영화프랑스 영화영국 영화독일 영화포커스 ...,로만 폴란스키 감독 장편 연출 작품 펼치기 접기 피아니스트의 주요 수상 이력...
1,제차 세계 대전,제차 세계 대전,제차 세계 대전 관련 틀 펼치기 접기 제차 세계 대전 참전국 펼치기 접기 관...
2,김영남,최고인민회의 상임위원장북한 외무상조선로동당 국제비서조선로동당 기 정치국 상무위원조선...,은는 여기로 연결됩니다 동명이인에 대한 내용은 김영남동명이인 문서를의 번 문단을...


In [32]:
title_corpus = ''.join(df['title'].tolist())
category_corpus = ''.join(df['category'].tolist())
content_corpus = ''.join(df['content'].tolist())

In [34]:
print(title_corpus[:300])

피아니스트로만 폴란스키 영화 제차 세계 대전 김영남 휴스턴 애스트로스년  도쿄 올림픽축구남자조 비누 가면라이더 리바이  도쿄 올림픽대한민국펜싱  스타팅 포켓몬신오지방 현하 털실 커비 이야기스테이지 퉁이 라이온스 데빈 렌쉬 가면라이더 리바이스제품 틀해군본부 정주영요리연구가 블리츠크랭크 김민우투수 콜 오브 듀티 모바일문제점 액티비전 블리자드 사내 성차별 및 성추행 논란  도쿄 올림픽대한민국태권도 나무위키프로젝트이미지 벡터화성과 아브렐슈드군단장 레이드  도쿄 올림픽대한민국 라테일시나리오 정은경 법륜승려비판 너프장난감종류 틀조국인물 카트라


In [36]:
!pip install konlpy
from konlpy.tag import Okt
from collections import Counter

  Using cached konlpy-0.5.2-py2.py3-none-any.whl (19.4 MB)
  Using cached JPype1-1.3.0-cp37-cp37m-macosx_10_9_x86_64.whl (381 kB)
  Using cached beautifulsoup4-4.6.0-py3-none-any.whl (86 kB)
  Using cached tweepy-3.10.0-py2.py3-none-any.whl (30 kB)
  Using cached requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)
  Using cached oauthlib-3.1.1-py2.py3-none-any.whl (146 kB)
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.9.3
    Uninstalling beautifulsoup4-4.9.3:
      Successfully uninstalled beautifulsoup4-4.9.3


In [37]:
nouns_tagger = Okt()
nouns = nouns_tagger.nouns(content_corpus)
count = Counter(nouns)
count

Counter({'로만': 29,
         '폴란스키': 11,
         '감독': 108,
         '장편': 3,
         '연출': 23,
         '작품': 77,
         '접기': 231,
         '피아니스트': 33,
         '주요': 94,
         '수상': 35,
         '이력': 3,
         '선정': 22,
         '세기': 35,
         '영화': 89,
         '년': 1193,
         '명의': 49,
         '국제': 254,
         '평론가': 5,
         '대상': 73,
         '사상': 50,
         '최고': 126,
         '편': 143,
         '위': 1148,
         '홀랜드': 3,
         '드라이브': 23,
         '화양연화': 3,
         '위데': 3,
         '윌': 7,
         '비': 147,
         '블러드': 3,
         '치히로': 3,
         '행방불명': 9,
         '보이': 70,
         '후드': 29,
         '터널': 12,
         '선샤인': 3,
         '트리': 66,
         '오브': 112,
         '라이프': 12,
         '하나': 274,
         '둘': 86,
         '위씨': 3,
         '민': 15,
         '나데르': 3,
         '거위': 3,
         '노인': 20,
         '나라': 82,
         '위인': 33,
         '사이드': 39,
         '르윈': 3,
         '조디악': 3,
         '위칠': 3,
    